<a href="https://colab.research.google.com/github/alexander-toschev/ai-tools/blob/main/tasks/Task6_FakeAnalysisPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
!pip install --upgrade gspread pandas google-auth
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
from IPython.display import display
import random
# Authenticate and create the PyDrive client.
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# FILL THIS
student_name = "ELON MUSK"
group_id = "11-101"

In [ ]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1Kfxj2eDFl7xQnXw7Fpb9bwghc65o8xf--VpNxrdWHaY/edit?gid=0#gid=0"
sh = gc.open_by_url(SPREADSHEET_URL)
worksheet = sh.sheet1

# Ensure header row exists
if not worksheet.get_all_values():
    worksheet.append_row(["Student Name", "Group","TaskID", "Score"])


# MAIN NOTEBOOK GOES HERE
task_id = "Task6_FakeAnalysisPipeline"
score = 0
max_score = 25

In [ ]:
# 🎓 Домашнее задание: Проверка подлинности видео с использованием видео- и аудиопотока

In [ ]:
# 0. Установка зависимостей
!pip install ffmpeg-python resemblyzer scikit-learn tensorflow matplotlib tqdm librosa opencv-python-headless

In [ ]:

# 1. Импорты
import ffmpeg
import os
import librosa
import numpy as np
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd
import cv2

In [ ]:


# Загрузка обученной модели на LA
model = load_model("la_model.keras")  # обучается отдельно
encoder = VoiceEncoder()

# Путь к видеофайлам
VIDEO_DIR = "/content/videos"
VIDEO_LIST = [os.path.join(VIDEO_DIR, f"video{i}.mp4") for i in range(1, 7)]

In [ ]:
# Извлечение аудио из видео
def extract_audio_from_video(video_path, output_wav):
    (
        ffmpeg
        .input(video_path)
        .output(output_wav, ac=1, ar=16000)
        .overwrite_output()
        .run(quiet=True)
    )


In [ ]:

# Предсказание с помощью модели
def predict_audio_model(file_path):
    wav = preprocess_wav(Path(file_path))
    # YOU CODE HERE USING THE LA model and resembler predict the video
    embed =
    prediction =
    return prediction, prediction >= 0.5

In [ ]:

# Эвристический анализ
def extract_simple_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    # YOu code starts here
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    mfcc_std =
    zcr =
    centroid =
    # YOUR CODE ENDS HERE
    return {
        "mfcc_std_mean": np.mean(mfcc_std),
        "zcr_mean": np.mean(zcr),
        "centroid_mean": np.mean(centroid)
    }

In [ ]:
def simple_fake_audio_detector(file_path):
    f = extract_simple_features(file_path)
    is_fake = f["mfcc_std_mean"] < 25 or f["zcr_mean"] > 0.1 or f["centroid_mean"] > 3500
    return f, is_fake

In [ ]:
# Анализ видеопотока с помощью FFException-подхода
IMG_SIZE = 299
MODEL_PATH = 'xception_dfdc_trained.keras'  # Предобученная модель

# === Загрузка обученной модели ===
model_exception = tf.keras.models.load_model(MODEL_PATH)


# === Преобразование для входа ===
def preprocess_face(face):
    face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
    face = face.astype('float32') / 255.0
    return np.expand_dims(face, axis=0)

# === Извлечение лица ===
def extract_face(frame):
    face_locations = face_recognition.face_locations(frame)
    if not face_locations:
        return None
    top, right, bottom, left = face_locations[0]
    face = frame[top:bottom, left:right]
    return face

def analyze_video_stream(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    predictions = []
    frame_count = 0

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        face = extract_face(frame)
        if face is None:
            continue
        # your code starts here
        input_face =
        pred =
        predictions.append(pred)
        frame_count += 1
    cap.release()
    if predictions:
        return "Fake detected",np.mean(predictions) > 0.5
    else:
        return None, False

In [ ]:







# Основной анализ одного видеофайла

def analyze_video_file(video_path):
    print(f"\n📂 Файл: {os.path.basename(video_path)}")
    audio_path = video_path.replace(".mp4", ".wav")
    extract_audio_from_video(video_path, audio_path)
    visual_log, visual_flag = analyze_video_stream(video_path)
    model_score, model_label = predict_audio_model(audio_path)
    heuristics, heur_label = simple_fake_audio_detector(audio_path)

    print("🎥 Видео (по кадрам):", visual_log, "⛔" if visual_flag else "✅")
    print("🔊 Аудио (Resemblyzer+NN):", "Fake" if model_label else "Real", f"(score={model_score:.3f})")
    print("🔎 Аудио (эвристика):", "Fake" if heur_label else "Real", heuristics)

    return {
        "video": os.path.basename(video_path),
        "visual_flag": int(visual_flag),
        "nn_result": int(model_label),
        "heur_result": int(heur_label),
        "model_score": model_score,
        "heuristics": heuristics
    }




In [ ]:
# Запуск пайплайна на всех видео
results = []
for video_path in VIDEO_LIST:
    result = analyze_video_file(video_path)
    results.append(result)

# Тест структуры результатов

def test_pipeline_structure(results):
    assert len(results) == 6, "Должно быть 6 видео"
    for r in results:
        assert 'nn_result' in r and 'heur_result' in r and 'visual_flag' in r
        score += 5
        assert isinstance(r['model_score'], float)
        score += 10
        assert isinstance(r['heuristics'], dict)
        score += 10
    print("✅ Pipeline test passed.")

test_pipeline_structure(results)

In [ ]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
# Save the result to Google Sheets
from datetime import datetime

# Get current date and time
now = datetime.now()
timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
worksheet.append_row([student_name,group_id, task_id, score, timestamp])

print(f"Test completed! {student_name}, your score is {score}/{max_score}.")